In [23]:
## Read data
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.tokenize import  word_tokenize
from pandarallel import pandarallel
import pickle

import os
import numpy as np


In [11]:
#!pip install spacy

In [46]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

Available CPUs: 16


In [25]:
%%time
post_data_full = pd.read_csv('wallstreetbets_submissions_zstextract.csv')
post_data_full.shape

/opt/conda/lib/python3.7/site-packages/IPython/core/magics/execution.py:1335: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


CPU times: user 13.9 s, sys: 1.91 s, total: 15.8 s
Wall time: 15.8 s


(2218243, 14)

In [26]:
import datetime

post_data_full['timestamp'] = post_data_full['created_utc'].apply(datetime.datetime.fromtimestamp)
post_data_full['timestamp'].min()

Timestamp('2012-04-11 16:40:40')

In [4]:
def clean_text(text):
    # Remove all text between square brackets
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove all text between parentheses
    text = re.sub(r'\(.*?\)', '', text)
    
    # Remove all text between curly braces
    text = re.sub(r'\{.*?\}', '', text)
    
    # Remove all text between angle brackets
    text = re.sub(r'<.*?>', '', text)
    
    # Remove all text between double quotes
    #text = re.sub(r'"(.*?)"', '', text)
    
    # Remove all text between single quotes
    #text = re.sub(r"'(.*?)'", '', text)
    
    # Remove everything between **____**
    text = re.sub(r'\*\*(.*?)\*\*','',text)
    
    # Remove words followed by ^
    text = re.sub(r'(\^\w*)','',text)
    
    # Remove mentions, hashtags, links
    text = re.sub(r'\||:|\-|\n|(@\w+.*?)|(http\w\S+.*?)|(#\w+)','',text)
    
    # Remove all ^Your daily posts
    text = re.sub(r'^Your daily trading discussion.*|^We are now using AutoModerator.*|^Every time a new submission.*|^Trading discussion only.*','',text)
    
    # Remove all newlines, tabs, and extra whitespace
    text = re.sub(r'[\n\t]+', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    
    return text

In [27]:
%%time

pandarallel.initialize()
post_data_full['clean_text'] = post_data_full['body'].parallel_apply(lambda x: clean_text(str(x)))
post_data_full = post_data_full[['id','title','clean_text','author','timestamp','upvotes','num_comments','score']]
post_data_full.groupby(['clean_text'])['author'].count().sort_values(ascending=False).head(5)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 2.78 s, sys: 1.08 s, total: 3.86 s
Wall time: 10.5 s


clean_text
            1400263
nan          661710
&amp;;         1012
                648
Discuss.        509
Name: author, dtype: int64

In [28]:
## Creating a copy
df = post_data_full.copy()

## Spacy NER


In [12]:
import spacy 
from spacy import displacy

2023-03-07 07:40:03.107846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 07:40:15.473519: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 07:40:15.475639: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 07:40:15.475667: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [40]:
#!python -m spacy download en_core_web_lg

In [41]:
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [21]:
def get_org_entities(text):
    doc = nlp(str(text))
    org_entities = []
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            org_entities.append(ent.text)
    return org_entities



In [42]:
%%time

pandarallel.initialize()
df['title_ner_lg'] = df['title'].parallel_apply(get_org_entities)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 4.65 s, sys: 2.61 s, total: 7.25 s
Wall time: 33min 50s


In [43]:
%%time

df['text_ner_lg'] = df['clean_text'].parallel_apply(get_org_entities)

CPU times: user 3.26 s, sys: 1.86 s, total: 5.12 s
Wall time: 28min 51s


In [44]:
df1 = df[['title_ner_lg','text_ner_lg']]

df1.to_parquet('ner_lg.parquet')

In [45]:
pd.options.display.max_colwidth=5000
df.head(10)


,id,title,clean_text,author,timestamp,upvotes,num_comments,score,title_ner,text_ner,title_ner_lg,text_ner_lg
0,s4jw1,Earnings season is here. Place your bets.,"I know that /r/investing is a great place for congregating with fellow market gamblers but it's not *exactly* the right place to be. So rather than flooding it with gambling posts and epic win/loss screen shots I figured I'd make another sub for this specifically for speculating and such. We'll see if it works. Subscribe if you're interested.Anyways, upcomming earnings * AAPL 4/24* AMZN 4/25 * GOOG 4/12 * GRPN 5/14 * LNKD 5/4 * NFLX 4/23* PCLN 5/8 * ANF 5/16* GMCR 5/2 * LVS 5/4 There's a lis...",[deleted],2012-04-11 16:40:40,16,22,13,[],"[NFLX, ANF, Google]",[],[Google]
1,s6r57,"GOOG - beat estimates, price barely rises.",nan,[deleted],2012-04-12 20:37:31,5,0,2,[],[],[],[]
2,sd5ai,My poorly timed opening position for AAPL earnings bet,"So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things. But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet. Unfortunately for me I got greedy and subsequently got slapped on the wrist.Position Bought an OTM straddle with intentions on selling right before earnings. This way I don't care about the results and ride the volatility wave up to the wave's crest.* BTO AAPL May put @ 525 for ...",[deleted],2012-04-16 22:29:37,14,21,12,[],"[GOOG, OTM]",[],"[OTM, BTO, BTO]"
3,se66f,Anyone betting on VVUS and their potential approval of their Qnexa drug today?,"I'm normally a long, but I've created a secondary account with a small amount of money I am comfortable with losing for jumping on opportunities like this and learning from my successes . I've been waiting a couple weeks for a good opportunity, and this looks like it may be it. Lot of buzz going around the company for their drug Qnexa potentially being approved by the FDA, and a couple sources even went as far as to call it ""an easy double"".I'll be honest, I'm completely new to this, and my ...",[deleted],2012-04-17 13:41:30,1,0,1,[VVUS],[FDA],"[VVUS, Qnexa]","[Qnexa, FDA]"
4,sinsk,"After HGSI spikes 97%, will share price drop at market open?",nan,CheeseYogi,2012-04-19 23:21:16,5,0,4,[HGSI],[],[HGSI],[]
5,shtex,"EBAY posts higher 1Q net income and revenue, soars.",nan,secondhandsondek,2012-04-19 14:10:20,10,4,7,[EBAY],[],[EBAY],[]
6,skmub,AAPL Earnings bets?,"Today, when AAPL was heading for max pain at 575, I entered a May 575/585 call spread for 4.50, my plan is that I anticipate AAPL moves up on Monday then I will enter into a 585/575 put spread as a hedge if I can get it for less than 5.00, depending on Monday, I may go with different strikes, but that's going to be my earnings play, does anyone else have their bets in place?",[deleted],2012-04-21 00:54:33,7,5,7,[AAPL Earnings],[],[],[AAPL]
7,sohin,NFLX bear coming out of hibernation. Earnings tonight. Any takers?,nan,secondhandsondek,2012-04-23 18:00:52,10,13,7,[NFLX],[],[NFLX],[]
8,sqhht,AAPL - this is why we call it a gamble. I really need this. Anyone else playing it?,nan,[deleted],2012-04-24 20:07:33,6,7,3,[],[],[],[]
9,sqd7j,PNRA - Panera Breads earnings tonight (4/24),nan,secondhandsondek,2012-04-24 18:50:53,6,3,4,[],[],[PNRA - Panera Breads],[]


## Sentiment analysis

In [52]:
#!pip install vaderSentiment
#!pip install textblob

In [53]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [54]:
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [55]:
%%time

df['polarity'] = df.parallel_apply(lambda x: TextBlob(x['clean_text']).sentiment.polarity, axis=1)
df['subjectivity'] = df.parallel_apply(lambda x: TextBlob(x['clean_text']).sentiment.subjectivity, axis=1)

# Create a positive / negative sentiment variable
df['sentiment'] = np.where(df['polarity'].astype(float) > 0, 'Positive', 'Negative')
df['sentiment'] = np.where(df['polarity'].astype(float) == 0, 'Neutral', df['sentiment'])

df[['clean_text', 'sentiment', 'polarity', 'subjectivity']].head(5)

CPU times: user 14.3 s, sys: 5.09 s, total: 19.3 s
Wall time: 2min 1s


,clean_text,sentiment,polarity,subjectivity
0,"I know that /r/investing is a great place for congregating with fellow market gamblers but it's not *exactly* the right place to be. So rather than flooding it with gambling posts and epic win/loss screen shots I figured I'd make another sub for this specifically for speculating and such. We'll see if it works. Subscribe if you're interested.Anyways, upcomming earnings * AAPL 4/24* AMZN 4/25 * GOOG 4/12 * GRPN 5/14 * LNKD 5/4 * NFLX 4/23* PCLN 5/8 * ANF 5/16* GMCR 5/2 * LVS 5/4 There's a lis...",Positive,0.248413,0.512302
1,nan,Neutral,0.000000,0.000000
2,"So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things. But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet. Unfortunately for me I got greedy and subsequently got slapped on the wrist.Position Bought an OTM straddle with intentions on selling right before earnings. This way I don't care about the results and ride the volatility wave up to the wave's crest.* BTO AAPL May put @ 525 for ...",Positive,0.078247,0.419156
3,"I'm normally a long, but I've created a secondary account with a small amount of money I am comfortable with losing for jumping on opportunities like this and learning from my successes . I've been waiting a couple weeks for a good opportunity, and this looks like it may be it. Lot of buzz going around the company for their drug Qnexa potentially being approved by the FDA, and a couple sources even went as far as to call it ""an easy double"".I'll be honest, I'm completely new to this, and my ...",Positive,0.117980,0.535859
4,nan,Neutral,0.000000,0.000000


In [56]:
df['sentiment'].value_counts()

Neutral     2091403
Positive      85409
Negative      41431
Name: sentiment, dtype: int64

In [58]:
pd.options.display.max_colwidth=5000

df[['clean_text','title_ner_lg','text_ner_lg', 'sentiment', 'polarity', 'subjectivity']].head(5)

,clean_text,title_ner_lg,text_ner_lg,sentiment,polarity,subjectivity
0,"I know that /r/investing is a great place for congregating with fellow market gamblers but it's not *exactly* the right place to be. So rather than flooding it with gambling posts and epic win/loss screen shots I figured I'd make another sub for this specifically for speculating and such. We'll see if it works. Subscribe if you're interested.Anyways, upcomming earnings * AAPL 4/24* AMZN 4/25 * GOOG 4/12 * GRPN 5/14 * LNKD 5/4 * NFLX 4/23* PCLN 5/8 * ANF 5/16* GMCR 5/2 * LVS 5/4 There's a list of the ones I'm following, based on their previously significant moves postearnings. What are yours? And if you're betting, what's your move? I haven't decided my own yet, but I'll let you know when I do. Google is tempting, but there's so much volatility built in that I am just tempted to be price neutral and sell volatility... We'll see.",[],[Google],Positive,0.248413,0.512302
1,nan,[],[],Neutral,0.000000,0.000000
2,"So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things. But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet. Unfortunately for me I got greedy and subsequently got slapped on the wrist.Position Bought an OTM straddle with intentions on selling right before earnings. This way I don't care about the results and ride the volatility wave up to the wave's crest.* BTO AAPL May put @ 525 for $10.10* BTO AAPL May call @ 665 $7.60It was poorly timed, because I tried to time the swings today hoping to trap some profits inside of the straddle . I bought the call during a price dip and I waited for prices to go back up before buying the put, which never really happened. That said, I . Any other AAPL takers?",[],"[OTM, BTO, BTO]",Positive,0.078247,0.419156
3,"I'm normally a long, but I've created a secondary account with a small amount of money I am comfortable with losing for jumping on opportunities like this and learning from my successes . I've been waiting a couple weeks for a good opportunity, and this looks like it may be it. Lot of buzz going around the company for their drug Qnexa potentially being approved by the FDA, and a couple sources even went as far as to call it ""an easy double"".I'll be honest, I'm completely new to this, and my sole decision to jump in on this stock is for the potential big ups that will happen today or tomorrow on that announcement plus the hype from around the web on it, then I'll be getting back out. I'll also be selling to cut losses if I need to as well.Thoughts?","[VVUS, Qnexa]","[Qnexa, FDA]",Positive,0.117980,0.535859
4,nan,[HGSI],[],Neutral,0.000000,0.000000


In [61]:
df1 = df[['sentiment','sentiment_title', 'polarity', 'subjectivity', 'polarity_title','subjectivity_title']]

df1.to_parquet('sentiment_title_text.parquet')

In [60]:
%%time

df['polarity_title'] = df.parallel_apply(lambda x: TextBlob(str(x['title'])).sentiment.polarity, axis=1)
df['subjectivity_title'] = df.parallel_apply(lambda x: TextBlob(str(x['title'])).sentiment.subjectivity, axis=1)

# Create a positive / negative sentiment variable
df['sentiment_title'] = np.where(df['polarity_title'].astype(float) > 0, 'Positive', 'Negative')
df['sentiment_title'] = np.where(df['polarity_title'].astype(float) == 0, 'Neutral', df['sentiment'])

df[['title','clean_text','title_ner_lg','text_ner_lg', 'sentiment','sentiment_title', 'polarity', 'subjectivity']].head(5)

CPU times: user 15.4 s, sys: 6.14 s, total: 21.5 s
Wall time: 2min 7s


,title,clean_text,title_ner_lg,text_ner_lg,sentiment,sentiment_title,polarity,subjectivity
0,Earnings season is here. Place your bets.,"I know that /r/investing is a great place for congregating with fellow market gamblers but it's not *exactly* the right place to be. So rather than flooding it with gambling posts and epic win/loss screen shots I figured I'd make another sub for this specifically for speculating and such. We'll see if it works. Subscribe if you're interested.Anyways, upcomming earnings * AAPL 4/24* AMZN 4/25 * GOOG 4/12 * GRPN 5/14 * LNKD 5/4 * NFLX 4/23* PCLN 5/8 * ANF 5/16* GMCR 5/2 * LVS 5/4 There's a list of the ones I'm following, based on their previously significant moves postearnings. What are yours? And if you're betting, what's your move? I haven't decided my own yet, but I'll let you know when I do. Google is tempting, but there's so much volatility built in that I am just tempted to be price neutral and sell volatility... We'll see.",[],[Google],Positive,Neutral,0.248413,0.512302
1,"GOOG - beat estimates, price barely rises.",nan,[],[],Neutral,Neutral,0.000000,0.000000
2,My poorly timed opening position for AAPL earnings bet,"So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things. But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet. Unfortunately for me I got greedy and subsequently got slapped on the wrist.Position Bought an OTM straddle with intentions on selling right before earnings. This way I don't care about the results and ride the volatility wave up to the wave's crest.* BTO AAPL May put @ 525 for $10.10* BTO AAPL May call @ 665 $7.60It was poorly timed, because I tried to time the swings today hoping to trap some profits inside of the straddle . I bought the call during a price dip and I waited for prices to go back up before buying the put, which never really happened. That said, I . Any other AAPL takers?",[],"[OTM, BTO, BTO]",Positive,Positive,0.078247,0.419156
3,Anyone betting on VVUS and their potential approval of their Qnexa drug today?,"I'm normally a long, but I've created a secondary account with a small amount of money I am comfortable with losing for jumping on opportunities like this and learning from my successes . I've been waiting a couple weeks for a good opportunity, and this looks like it may be it. Lot of buzz going around the company for their drug Qnexa potentially being approved by the FDA, and a couple sources even went as far as to call it ""an easy double"".I'll be honest, I'm completely new to this, and my sole decision to jump in on this stock is for the potential big ups that will happen today or tomorrow on that announcement plus the hype from around the web on it, then I'll be getting back out. I'll also be selling to cut losses if I need to as well.Thoughts?","[VVUS, Qnexa]","[Qnexa, FDA]",Positive,Neutral,0.117980,0.535859
4,"After HGSI spikes 97%, will share price drop at market open?",nan,[HGSI],[],Neutral,Neutral,0.000000,0.000000


In [ ]:
## Sentiment Vader

In [62]:
df.shape[0]

2218243

In [64]:
xtmp = pd.read_csv('vader_text.csv')
xtmp_1 = pd.read_csv('vader_title.csv')


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


NameError: name 'xtmp1' is not defined

In [65]:

print(xtmp.shape[0],' ',xtmp_1.shape[0])

2218248   2218248


In [71]:
xtmp_1.head()

,id,clean_title,clean_text,author,timestamp,upvotes,num_comments,score,len,neg,neu,pos,compound,sentiment
0,s4jw1,earnings season is here. place your bets.,"I know that /r/investing is a great place for congregating with fellow market gamblers but it's not *exactly* the right place to be. So rather than flooding it with gambling posts and epic win/loss screen shots I figured I'd make another sub for this specifically for speculating and such. We'll see if it works. Subscribe if you're interested.Anyways, upcomming earnings * AAPL 4/24* AMZN 4/25 * GOOG 4/12 * GRPN 5/14 * LNKD 5/4 * NFLX 4/23* PCLN 5/8 * ANF 5/16* GMCR 5/2 * LVS 5/4 There's a list of the ones I'm following, based on their previously significant moves postearnings. What are yours? And if you're betting, what's your move? I haven't decided my own yet, but I'll let you know when I do. Google is tempting, but there's so much volatility built in that I am just tempted to be price neutral and sell volatility... We'll see.",[deleted],2012-04-11 11:40:40,16.0,22.0,13.0,7.0,0.000,1.000,0.0,0.0000,Neutral
1,s6r57,"goog beat estimates, price barely rises.",NaN,[deleted],2012-04-12 15:37:31,5.0,0.0,2.0,6.0,0.000,1.000,0.0,0.0000,Neutral
2,sd5ai,my poorly timed opening position for aapl earnings bet,"So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things. But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet. Unfortunately for me I got greedy and subsequently got slapped on the wrist.Position Bought an OTM straddle with intentions on selling right before earnings. This way I don't care about the results and ride the volatility wave up to the wave's crest.* BTO AAPL May put @ 525 for $10.10* BTO AAPL May call @ 665 $7.60It was poorly timed, because I tried to time the swings today hoping to trap some profits inside of the straddle . I bought the call during a price dip and I waited for prices to go back up before buying the put, which never really happened. That said, I . Any other AAPL takers?",[deleted],2012-04-16 17:29:37,14.0,21.0,12.0,9.0,0.000,1.000,0.0,0.0000,Neutral
3,se66f,anyone betting on vvus and their potential approval of their qnexa drug today?,"I'm normally a long, but I've created a secondary account with a small amount of money I am comfortable with losing for jumping on opportunities like this and learning from my successes . I've been waiting a couple weeks for a good opportunity, and this looks like it may be it. Lot of buzz going around the company for their drug Qnexa potentially being approved by the FDA, and a couple sources even went as far as to call it ""an easy double"".I'll be honest, I'm completely new to this, and my sole decision to jump in on this stock is for the potential big ups that will happen today or tomorrow on that announcement plus the hype from around the web on it, then I'll be getting back out. I'll also be selling to cut losses if I need to as well.Thoughts?",[deleted],2012-04-17 08:41:30,1.0,0.0,1.0,13.0,0.000,0.795,0.205,0.4767,Positive
4,sinsk,"after hgsi spikes 97%, will share price drop at market open?",NaN,CheeseYogi,2012-04-19 18:21:16,5.0,0.0,4.0,11.0,0.158,0.677,0.165,0.0258,Positive


In [72]:
df.head()

,id,title,clean_text,author,timestamp,upvotes,num_comments,score,title_ner,text_ner,title_ner_lg,text_ner_lg,polarity,subjectivity,sentiment,polarity_title,subjectivity_title,sentiment_title
0,s4jw1,Earnings season is here. Place your bets.,"I know that /r/investing is a great place for congregating with fellow market gamblers but it's not *exactly* the right place to be. So rather than flooding it with gambling posts and epic win/loss screen shots I figured I'd make another sub for this specifically for speculating and such. We'll see if it works. Subscribe if you're interested.Anyways, upcomming earnings * AAPL 4/24* AMZN 4/25 * GOOG 4/12 * GRPN 5/14 * LNKD 5/4 * NFLX 4/23* PCLN 5/8 * ANF 5/16* GMCR 5/2 * LVS 5/4 There's a list of the ones I'm following, based on their previously significant moves postearnings. What are yours? And if you're betting, what's your move? I haven't decided my own yet, but I'll let you know when I do. Google is tempting, but there's so much volatility built in that I am just tempted to be price neutral and sell volatility... We'll see.",[deleted],2012-04-11 16:40:40,16,22,13,[],"[NFLX, ANF, Google]",[],[Google],0.248413,0.512302,Positive,0.00,0.0,Neutral
1,s6r57,"GOOG - beat estimates, price barely rises.",nan,[deleted],2012-04-12 20:37:31,5,0,2,[],[],[],[],0.000000,0.000000,Neutral,0.05,0.1,Neutral
2,sd5ai,My poorly timed opening position for AAPL earnings bet,"So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things. But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet. Unfortunately for me I got greedy and subsequently got slapped on the wrist.Position Bought an OTM straddle with intentions on selling right before earnings. This way I don't care about the results and ride the volatility wave up to the wave's crest.* BTO AAPL May put @ 525 for $10.10* BTO AAPL May call @ 665 $7.60It was poorly timed, because I tried to time the swings today hoping to trap some profits inside of the straddle . I bought the call during a price dip and I waited for prices to go back up before buying the put, which never really happened. That said, I . Any other AAPL takers?",[deleted],2012-04-16 22:29:37,14,21,12,[],"[GOOG, OTM]",[],"[OTM, BTO, BTO]",0.078247,0.419156,Positive,-0.40,0.6,Positive
3,se66f,Anyone betting on VVUS and their potential approval of their Qnexa drug today?,"I'm normally a long, but I've created a secondary account with a small amount of money I am comfortable with losing for jumping on opportunities like this and learning from my successes . I've been waiting a couple weeks for a good opportunity, and this looks like it may be it. Lot of buzz going around the company for their drug Qnexa potentially being approved by the FDA, and a couple sources even went as far as to call it ""an easy double"".I'll be honest, I'm completely new to this, and my sole decision to jump in on this stock is for the potential big ups that will happen today or tomorrow on that announcement plus the hype from around the web on it, then I'll be getting back out. I'll also be selling to cut losses if I need to as well.Thoughts?",[deleted],2012-04-17 13:41:30,1,0,1,[VVUS],[FDA],"[VVUS, Qnexa]","[Qnexa, FDA]",0.117980,0.535859,Positive,0.00,1.0,Neutral
4,sinsk,"After HGSI spikes 97%, will share price drop at market open?",nan,CheeseYogi,2012-04-19 23:21:16,5,0,4,[HGSI],[],[HGSI],[],0.000000,0.000000,Neutral,0.00,0.5,Neutral


In [ ]:
x1 = xtmp[['neg','neu','pos','compound','sentiment']]
x2 = xtmp1[['neg','neu','pos','compound','sentiment']]

x1.to_parquet('sentiment_vader_text.parquet')
x2.to_parquet('sentiment_vader_title.parquet')